In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
rawdat = pd.concat([pd.read_csv('../data/input/Auschwitz_segments_03112020_1.csv'),
                   pd.read_csv('../data/input/Auschwitz_segments_03112020_2.csv')])
rawdat.shape

(1067267, 12)

In [3]:
biodata = pd.read_csv('../data/input/biodata_birkenau.csv')

In [4]:
# filter only used interviews from biodata_birkenau.csv
relevant_rawdat = rawdat[rawdat.IntCode.isin(biodata['IntCode'])]
relevant_rawdat.shape

(584447, 12)

In [5]:
# create a keyword ID - Label map
kwID_kwLabel_map = relevant_rawdat[['KeywordID', 'KeywordLabel']]
kwID_kwLabel_map.drop_duplicates(inplace=True)
kwID_kwLabel_map.reset_index(inplace=True, drop=True)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
kwID_kwLabel_map

,KeywordID,KeywordLabel
0,7601,Auschwitz II-Birkenau (Poland : Death Camp)
1,13310,"Oświęcim (Kraków, Poland)"
2,14226,Poland 1944 (July 22) - 1945 (January 16)
3,12044,camp selections
4,14280,loved ones' separations
...,...,...
2334,83352,Christians
2335,59509,European history
2336,15692,attitudes toward Canada and/or Canadians
2337,62391,attitudes toward socialism and/or socialists


In [7]:
# find kwIds, that are in the first and last segments of relevant interviews
first_kwIDs = []
last_kwIDs = []

for intcode in pd.unique(relevant_rawdat.IntCode):
    interview = relevant_rawdat[relevant_rawdat.IntCode==intcode]
    
    # identify first and alst segments
    # extract KwID
    first_kwID = interview[interview.SegmentNumber == interview.SegmentNumber.min()].KeywordID
    last_kwID = interview[interview.SegmentNumber == interview.SegmentNumber.max()].KeywordID
    print (first_kwID)
    first_kwIDs.append(first_kwID.to_numpy())
    last_kwIDs.append(last_kwID.to_numpy())
    
# concatenate
first_kwIDs = np.concatenate(first_kwIDs)
last_kwIDs = np.concatenate(last_kwIDs)

0     7601
1    13310
2    14226
Name: KeywordID, dtype: int64
43     7601
44    13310
45    14233
Name: KeywordID, dtype: int64
153     7601
154    12044
155    16503
156    13310
157    14233
158    25452
Name: KeywordID, dtype: int64
179     7601
180    10852
181    16328
182    14049
183    13310
Name: KeywordID, dtype: int64
428     7601
429    13310
430    14233
Name: KeywordID, dtype: int64
506     7601
507    13310
508    14233
Name: KeywordID, dtype: int64
606     7601
607    10983
608    13310
609    14233
Name: KeywordID, dtype: int64
729     7601
730    10983
731    16328
732    13310
733    14233
Name: KeywordID, dtype: int64
968     7601
969    13310
970    14233
Name: KeywordID, dtype: int64
1127     7601
1128    12044
1129    14280
1130    13310
1131    14233
Name: KeywordID, dtype: int64
1271     7601
1272    13310
1273    14233
Name: KeywordID, dtype: int64
1392     7601
1393    13310
1394    14233
Name: KeywordID, dtype: int64
1479     7601
1480    13310
1481    1423

20184     7528
20185    14280
20186    13310
20187    14233
Name: KeywordID, dtype: int64
20727     7601
20728    13310
20729    14233
Name: KeywordID, dtype: int64
20903     7601
20904    10983
20905    13310
20906    14233
Name: KeywordID, dtype: int64
20992     7601
20993    10983
20994    13310
20995    14233
Name: KeywordID, dtype: int64
21313     7601
21314    13310
21315    14233
21316    16345
Name: KeywordID, dtype: int64
21555     7601
21556    13310
21557    14233
Name: KeywordID, dtype: int64
21874     7601
21875    12044
21876    16328
21877    13310
21878    14233
Name: KeywordID, dtype: int64
21958     7528
21959    10983
21960    13310
21961    14233
Name: KeywordID, dtype: int64
22044     7601
22045    10983
22046    14049
22047    13310
22048    14233
Name: KeywordID, dtype: int64
22131     7601
22132    13310
22133    14233
Name: KeywordID, dtype: int64
22189     7528
22190    10983
22191    16123
22192    13310
22193    14226
Name: KeywordID, dtype: int64
22228     

40366     7601
40367    13310
40368    14233
Name: KeywordID, dtype: int64
40417     7601
40418    12044
40419    13310
40420    14233
Name: KeywordID, dtype: int64
40859     7601
40860    13214
40861    16328
40862    13310
Name: KeywordID, dtype: int64
40915     7601
40916    26611
40917    13310
40918    14233
40919     4083
Name: KeywordID, dtype: int64
40994     7528
40995    10983
40996    13310
40997    14233
Name: KeywordID, dtype: int64
41075     7601
41076    13310
41077    14233
Name: KeywordID, dtype: int64
41263     7601
41264    13310
41265    14233
Name: KeywordID, dtype: int64
41288     7601
41289    10983
41290    15456
41291    13310
41292    38840
41293    14233
Name: KeywordID, dtype: int64
41384     7601
41385    10853
41386    13310
41387    14233
Name: KeywordID, dtype: int64
41480     7601
41481    10852
41482    13310
41483    14233
Name: KeywordID, dtype: int64
41837     7601
41838    13310
41839    14233
Name: KeywordID, dtype: int64
41960     7601
41961    1

61498     7601
61499    13310
61500    14233
Name: KeywordID, dtype: int64
61534     7601
61535    13310
61536    14233
61537    16192
Name: KeywordID, dtype: int64
61625     7601
61626    16503
61627    26611
61628    13310
61629    14233
Name: KeywordID, dtype: int64
61749     7601
61750    10983
61751    13310
61752    14233
Name: KeywordID, dtype: int64
62175     7601
62176    12044
62177     4047
62178    13310
62179    14233
Name: KeywordID, dtype: int64
62432     7601
62433    13310
62434    14233
Name: KeywordID, dtype: int64
62574     7601
62575    13310
62576    14233
Name: KeywordID, dtype: int64
62617     7601
62618    13310
62619    14233
Name: KeywordID, dtype: int64
62687     7601
62688    10983
62689    13310
62690    14233
Name: KeywordID, dtype: int64
62869     7601
62870    13310
62871    14233
Name: KeywordID, dtype: int64
62929     7601
62930    13310
62931    14233
Name: KeywordID, dtype: int64
63161     7601
63162    12044
63163    13310
63164    14233
Name: Keyw

84619     7601
84620    10983
84621    13310
84622    14233
Name: KeywordID, dtype: int64
84701     7601
84702    10983
84703    13310
84704    14233
Name: KeywordID, dtype: int64
84831     7601
84832    12044
84833    13310
84834    14233
Name: KeywordID, dtype: int64
84903     7601
84904    10853
84905    13310
84906    14233
Name: KeywordID, dtype: int64
84996     7601
84997    10983
84998    13310
84999    14233
Name: KeywordID, dtype: int64
85132     7601
85133    10983
85134    13310
85135    14233
Name: KeywordID, dtype: int64
85369     7601
85370    13310
85371    14233
Name: KeywordID, dtype: int64
85515     7528
85516    12044
85517    13310
85518    14233
Name: KeywordID, dtype: int64
85573     7601
85574    11672
85575    14280
85576    13310
85577    14233
Name: KeywordID, dtype: int64
85641     7601
85642    13310
85643    14233
85644    12161
Name: KeywordID, dtype: int64
85712     7601
85713    16856
85714    16328
85715    13310
85716    14233
Name: KeywordID, dtype: i

...

452746     7601
452747    11672
452748    14280
452749    13310
452750    14233
452751    16277
Name: KeywordID, dtype: int64
452982     7601
452983    13310
452984    14233
Name: KeywordID, dtype: int64
453081     7601
453082    12044
453083    13310
453084    14233
Name: KeywordID, dtype: int64
453115     7601
453116    13310
453117    14233
Name: KeywordID, dtype: int64
453340     7601
453341    14280
453342    13310
453343    14233
453344    46670
Name: KeywordID, dtype: int64
453376     7601
453377    12044
453378     4047
453379    13310
453380    14233
Name: KeywordID, dtype: int64
453441     7601
453442    10983
453443    15337
453444    13310
453445    14233
Name: KeywordID, dtype: int64
453510     7601
453511    12044
453512    13310
453513    14232
453514     6861
Name: KeywordID, dtype: int64
453585     7601
453586    10983
453587    13310
453588    14233
Name: KeywordID, dtype: int64
453685     7528
453686    11672
453687    14280
453688    13310
453689    14233
Name: Keyw

488493     7528
488494    16103
488495    16123
488496    15223
488497    15774
488498     4889
488499     8057
488500    12754
488501    27440
488502    13310
488503    14233
Name: KeywordID, dtype: int64
488803     7528
488804    16451
488805    26399
488806    14280
488807    13310
488808    14233
Name: KeywordID, dtype: int64
489002     7601
489003    10983
489004    12288
489005    13310
489006    14233
489007    10005
Name: KeywordID, dtype: int64
489076     7601
489077    10853
489078    12044
489079    11672
489080    14280
489081    13310
489082    14233
Name: KeywordID, dtype: int64
489155     7601
489156    10853
489157    13310
489158    14233
Name: KeywordID, dtype: int64
489507     7601
489508    12044
489509     6777
489510    11672
489511    14280
489512     4047
489513    13310
489514    14226
Name: KeywordID, dtype: int64
489680     7528
489681    10853
489682    12126
489683    14280
489684    13310
489685    14233
Name: KeywordID, dtype: int64
489908     7601
489909

503195     7601
503196    14380
503197    10853
503198    30447
503199    13310
503200    14233
Name: KeywordID, dtype: int64
503477     7601
503478    10983
503479    13310
503480    14233
Name: KeywordID, dtype: int64
503613     7528
503614    10853
503615     3242
503616    14280
503617    13310
503618    14233
Name: KeywordID, dtype: int64
503656     7601
503657    10853
503658    11015
503659    10698
503660    13310
503661    14233
Name: KeywordID, dtype: int64
503808     7601
503809    16451
503810    10983
503811    13875
503812    10853
503813    15585
503814     4227
503815    11672
503816    13310
503817    14233
Name: KeywordID, dtype: int64
504045     7601
504046    10983
504047    13310
504048    14233
Name: KeywordID, dtype: int64
504300     7601
504301    16103
504302    16328
504303    15774
504304    12127
504305    12328
504306     8057
504307    12754
504308    13310
504309    14233
Name: KeywordID, dtype: int64
504417     7528
504418    10852
504419    16103
504420

514312     7601
514313     4227
514314    13310
514315    14233
514316    14917
514317    25452
Name: KeywordID, dtype: int64
514559     7601
514560    10853
514561    13310
514562    14233
Name: KeywordID, dtype: int64
514617     7601
514618    33502
514619    16328
514620    15223
514621    15774
514622    11713
514623    14049
514624    14541
514625    12720
514626    13310
514627    14233
Name: KeywordID, dtype: int64
515148     7601
515149    15774
515150    13310
515151    19670
515152    22937
515153    14233
515154    19672
515155    16345
515156    15232
Name: KeywordID, dtype: int64
515241     7601
515242    10983
515243    21332
515244    14293
515245    13310
515246    14233
515247    14226
Name: KeywordID, dtype: int64
515307     7601
515308     7991
515309    13363
515310    10852
515311    16848
515312    16328
515313    15223
515314    15774
515315    13310
515316    14233
515317    14307
Name: KeywordID, dtype: int64
515765     7528
515766    15337
515767    13310
5157

530601     7601
530602    15098
530603    10853
530604    13214
530605    13310
530606    14233
Name: KeywordID, dtype: int64
530652     7601
530653    10698
530654    13310
530655    14233
Name: KeywordID, dtype: int64
530692    13018
530693    13310
530694    14232
Name: KeywordID, dtype: int64
531430     7528
531431    10983
531432    13310
531433    14232
Name: KeywordID, dtype: int64
531705     7601
531706    10983
531707    13310
531708    14233
Name: KeywordID, dtype: int64
532127     7528
532128    10853
532129    13310
532130    14232
Name: KeywordID, dtype: int64
532366     7601
532367    12044
532368    13310
532369    14233
Name: KeywordID, dtype: int64
532554     7601
532555    10983
532556    13310
532557    14233
Name: KeywordID, dtype: int64
532644     7601
532645    14370
532646    12122
532647    13310
532648    14233
532649    23423
532650    11786
Name: KeywordID, dtype: int64


In [49]:
# bincount (histogram topics in both cases)
bc_first = np.bincount(first_kwIDs, minlength=kwID_kwLabel_map.KeywordID.max())
bc_last = np.bincount(last_kwIDs, minlength=kwID_kwLabel_map.KeywordID.max())

In [74]:
# argsort (backwards) to get highest populated states
idx_first = bc_first.argsort()[::-1]
idx_last = bc_last.argsort()[::-1]

# printed number of topics (only printout)
print_top_n = 15

In [76]:
# left is prevalence of a topic, i.e. fraction of first frames that have this topic
# second is topic, followed by keywordID in brackets.
print('first topics')
for idx in idx_first[:print_top_n]:
    print(f'{np.round(bc_first[idx]/bc_first.sum(), 2)}: '+\
          f'{kwID_kwLabel_map[kwID_kwLabel_map.KeywordID == idx].KeywordLabel.values[0]}' +\
          f'[KeywordID: {idx}]'
         )

first topics
0.23: Oświęcim (Kraków, Poland)[KeywordID: 13310]
0.21: Poland 1944[KeywordID: 14233]
0.18: Auschwitz II-Birkenau (Poland : Death Camp)[KeywordID: 7601]
0.05: Auschwitz (Poland : Concentration Camp)(generic)[KeywordID: 7528]
0.04: camp selections[KeywordID: 12044]
0.04: camp first impressions[KeywordID: 10983]
0.02: camp intake procedures[KeywordID: 10853]
0.02: deportation to Auschwitz II-Birkenau (Poland : Death Camp)[KeywordID: 16328]
0.02: loved ones' separations[KeywordID: 14280]
0.02: Poland 1943[KeywordID: 14232]
0.01: Poland 1944 (July 22) - 1945 (January 16)[KeywordID: 14226]
0.01: freight trains[KeywordID: 15774]
0.01: loved ones' final contacts[KeywordID: 11672]
0.01: Mengele, Josef[KeywordID: 4047]
0.0: deportation to Auschwitz (Poland : Concentration Camp)(generic)[KeywordID: 16123]


In [77]:
print('last topics')
for idx in idx_last[:print_top_n]:
    print(f'{np.round(bc_last[idx]/bc_last.sum(), 2)}: '+\
          f'{kwID_kwLabel_map[kwID_kwLabel_map.KeywordID == idx].KeywordLabel.values[0]}' +\
          f'[KeywordID: {idx}]'
         )

last topics
0.25: Oświęcim (Kraków, Poland)[KeywordID: 13310]
0.19: Auschwitz II-Birkenau (Poland : Death Camp)[KeywordID: 7601]
0.19: Poland 1944[KeywordID: 14233]
0.05: Auschwitz (Poland : Concentration Camp)(generic)[KeywordID: 7528]
0.03: transfer from Auschwitz II-Birkenau (Poland : Death Camp)[KeywordID: 16297]
0.02: Poland 1945 (January 1 - May 7)[KeywordID: 16192]
0.01: transfers, means of transport[KeywordID: 15232]
0.01: Germany 1944[KeywordID: 13929]
0.01: Poland 1944 (July 22) - 1945 (January 16)[KeywordID: 14226]
0.01: freight trains[KeywordID: 15774]
0.01: Poland 1943[KeywordID: 14232]
0.01: Germany 1945 (January 1 - May 7)[KeywordID: 13930]
0.01: camp selections[KeywordID: 12044]
0.01: transfer from Auschwitz (Poland : Concentration Camp)(generic)[KeywordID: 16162]
0.01: transfer conditions[KeywordID: 12161]
